## Accelerate Inference: Neural Network Pruning

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.9.2


In [2]:
# ! pip install --user install tensorflow==2.10.0 
# print(tf.version.VERSION)

In [2]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [3]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [87]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [5]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


703/703 [==============================] - 12s 5ms/step - loss: 1.5148 - accuracy: 0.3085 - val_loss: 1.3807 - val_accuracy: 0.4154
Epoch 2/50
703/703 [==============================] - 3s 5ms/step - loss: 1.3431 - accuracy: 0.4229 - val_loss: 1.2539 - val_accuracy: 0.4582
Epoch 3/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2655 - accuracy: 0.4639 - val_loss: 1.2748 - val_accuracy: 0.4352
Epoch 4/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2213 - accuracy: 0.4908 - val_loss: 1.1551 - val_accuracy: 0.5220
Epoch 5/50
703/703 [==============================] - 3s 5ms/step - loss: 1.1856 - accuracy: 0.5159 - val_loss: 1.1796 - val_accuracy: 0.5050
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1598 - accuracy: 0.5243 - val_loss: 1.1058 - val_accuracy: 0.5505
Epoch 7/50
703/703 [==============================] - 3s 5ms/step - loss: 1.1340 - accuracy: 0.5402 - val_loss: 1.0797 - val_accuracy: 0.5604
Epoch 8/50
703/7

In [6]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 10ms/step - loss: 0.7226 - accuracy: 0.7149


In [152]:
# perform pruning here

import copy

def prune(model, conv_prune_rate, dense_prune_rate):

    # Collect all convolutional layers
    conv_indices = [] 
    dense_indices = []
    c = len(model.layers)
    for i in range(c):
        name = model.layers[i].name
        if name.startswith("conv"):
            conv_indices.append(i)
        elif name.startswith("dense"):
            dense_indices.append(i)
    
    # Prune convolutions
    for i in conv_indices:

        layer = model.layers[i]   
        w, b = layer.get_weights()[0], layer.get_weights()[1]
        threshold = np.percentile(w, conv_prune_rate*100)
        w[w < threshold] = 0
        model.layers[i].set_weights([w,b])

    # Prune dense
    for i in dense_indices:

        layer = model.layers[i]
        w, b = layer.get_weights()[0], layer.get_weights()[1]
        threshold = np.percentile(w, dense_prune_rate*100)
        w[w < threshold] = 0
        model.layers[i].set_weights([w,b])

# get the weights 
# weights = model.get_weights()
# print(weights[0].shape)

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

# weights[7][:10]=0
# model.set_weights(weights)


In [170]:
# Clone model and set original weights
model2 = keras.models.clone_model(model)
model2.build() # replace 10 with number of variables in input layer
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.set_weights(model.get_weights())

# Prune
prune(model2, 0.1, 0)

# Count Stats
total_conv_weights = 0
c = len(model2.layers)
for i in range(c):
    name = model2.layers[i].name
    if(name.startswith("conv")):
      w1 = model2.layers[i].get_weights()[0]
      for weight in w1:
        total_conv_weights += weight.size

total_weights = 0
zero_weights = 0
all_model2_weights = model2.get_weights()
for weight in all_model2_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)


print(total_weights, total_conv_weights, zero_weights, zero_weights/total_weights)

592933 65376 6540 0.011029914003774456


In [154]:
# evaluate again to see how the accuracy changes
results = model2.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 5ms/step - loss: 36.8202 - accuracy: 0.1774


In [155]:
#train again with pruned

model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model2.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32


Epoch 1/50
703/703 [==============================] - 4s 5ms/step - loss: 2.1683 - accuracy: 0.2200 - val_loss: 1.6115 - val_accuracy: 0.1968
Epoch 2/50
703/703 [==============================] - 3s 5ms/step - loss: 1.6130 - accuracy: 0.2138 - val_loss: 1.6048 - val_accuracy: 0.2147
Epoch 3/50
703/703 [==============================] - 3s 5ms/step - loss: 1.6019 - accuracy: 0.2251 - val_loss: 1.5970 - val_accuracy: 0.2250
Epoch 4/50
703/703 [==============================] - 3s 5ms/step - loss: 1.5144 - accuracy: 0.3044 - val_loss: 1.1717 - val_accuracy: 0.5366
Epoch 5/50
703/703 [==============================] - 3s 5ms/step - loss: 1.0280 - accuracy: 0.5909 - val_loss: 0.8249 - val_accuracy: 0.6776
Epoch 6/50
703/703 [==============================] - 3s 5ms/step - loss: 0.8186 - accuracy: 0.6838 - val_loss: 0.8262 - val_accuracy: 0.6804
Epoch 7/50
703/703 [==============================] - 4s 5ms/step - loss: 0.7732 - accuracy: 0.7031 - val_loss: 0.8169 - val_accuracy: 0.6792
Epoch 

In [156]:
# evaluate again to see how the accuracy changes
results = model2.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 5ms/step - loss: 0.7196 - accuracy: 0.7354


In [165]:
# Repeat for dense layers
model3 = keras.models.clone_model(model2)
model3.build() # replace 10 with number of variables in input layer
model3.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model3.set_weights(model2.get_weights())

prune(model3, 0.0, 0.17)

c = len(model3.layers)
total_weights = 0
zero_weights = 0
all_model3_weights = model3.get_weights()
for weight in all_model3_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

print(total_weights, zero_weights, zero_weights/total_weights)

592933 89565 0.1510541663223332


In [166]:
# evaluate again to see how the accuracy changes
results = model3.evaluate(val_images, val_labels, batch_size=128)
print(results[1])
print((results[1] + zero_weights/total_weights)/2)

20/20 [==============================] - 0s 5ms/step - loss: 1.3704 - accuracy: 0.6127
0.612673282623291
0.38186372447281214


In [108]:
# Original model for comparison
total_weights = 0
zero_weights = 0
all_model_weights = model.get_weights()
for weight in all_model_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

print(total_weights, zero_weights)
print((0.7149 + zero_weights/total_weights)/2)

592933 0
0.36355


In [206]:
sparsity = [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]

# sparsity = [0, 0.10]

mag_spar = []
mag_acc = []

for s in sparsity:

  model_a = keras.models.clone_model(model)
  model_a.build() # replace 10 with number of variables in input layer
  model_a.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  model_a.set_weights(model.get_weights())
  prune(model_a, s, 0)

  model_a.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  history = model_a.fit(train_images, train_labels, batch_size=32, epochs=50, 
                      validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

  # for dense layers
  model_b = keras.models.clone_model(model_a)
  model_b.build() # replace 10 with number of variables in input layer
  model_b.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  model_b.set_weights(model_a.get_weights())
  prune(model_b, 0.0, 0.17)

  results = model_b.evaluate(val_images, val_labels, batch_size=128)
  mag_acc.append(results[1])

  total_weights = 0
  zero_weights = 0
  all_model_weights = model_b.get_weights()
  for weight in all_model_weights:
    total_weights += weight.size
    zero_weights += np.sum(weight == 0)

  mag_spar.append(zero_weights/total_weights)


Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


703/703 [==============================] - 5s 6ms/step - loss: 0.6385 - accuracy: 0.7594 - val_loss: 0.7379 - val_accuracy: 0.7050
Epoch 2/50
703/703 [==============================] - 4s 6ms/step - loss: 0.6256 - accuracy: 0.7645 - val_loss: 0.7265 - val_accuracy: 0.7232
Epoch 3/50
703/703 [==============================] - 4s 6ms/step - loss: 0.6255 - accuracy: 0.7607 - val_loss: 0.7254 - val_accuracy: 0.7236
Epoch 4/50
703/703 [==============================] - 4s 6ms/step - loss: 0.6146 - accuracy: 0.7700 - val_loss: 0.7400 - val_accuracy: 0.7081
Epoch 5/50
703/703 [==============================] - 4s 6ms/step - loss: 0.6111 - accuracy: 0.7697 - val_loss: 0.7140 - val_accuracy: 0.7208
Epoch 6/50
703/703 [==============================] - 4s 6ms/step - loss: 0.5962 - accuracy: 0.7748 - val_loss: 0.7134 - val_accuracy: 0.7251
Epoch 7/50
703/703 [==============================] - 4s 6ms/step - loss: 0.5954 - accuracy: 0.7779 - val_loss: 0.7166 - val_accuracy: 0.7236
Epoch 8/50
703/70

In [207]:

# try extremes
sparsity_fix = [s * total_conv_weights for s in sparsity]
total = [s * total_weights for s in mag_spar]
sparsity_done = [(total[i] + sparsity_fix[i]) / total_weights for i in range(len(total))]
# print(total)
print(sparsity_done)
print(mag_acc)

[0.1510541663223332, 0.1620800326512439, 0.17310589898015458, 0.18413176530906528, 0.19515763163797595, 0.20618349796688665, 0.21720936429579735, 0.22823523062470805, 0.2392610969536187, 0.2508081014212398]
[0.6348514556884766, 0.5988118648529053, 0.5061386227607727, 0.42099010944366455, 0.4332673251628876, 0.43643563985824585, 0.19207920134067535, 0.5611881017684937, 0.5429702997207642, 0.5140594244003296]


In [189]:
model_a_final = keras.models.clone_model(model)
model_a_final.build() # replace 10 with number of variables in input layer
model_a_final.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model_a_final.set_weights(model.get_weights())

prune(model_a_final, 0.1, 0)

model_a_final.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
history = model_a_final.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32


Epoch 1/50
703/703 [==============================] - 5s 6ms/step - loss: 2.1206 - accuracy: 0.2125 - val_loss: 1.6108 - val_accuracy: 0.1992
Epoch 2/50
703/703 [==============================] - 3s 5ms/step - loss: 1.6111 - accuracy: 0.2180 - val_loss: 1.6083 - val_accuracy: 0.2032
Epoch 3/50
703/703 [==============================] - 3s 5ms/step - loss: 1.6008 - accuracy: 0.2326 - val_loss: 1.5632 - val_accuracy: 0.2836
Epoch 4/50
703/703 [==============================] - 4s 5ms/step - loss: 1.4755 - accuracy: 0.3317 - val_loss: 1.1926 - val_accuracy: 0.5050
Epoch 5/50
703/703 [==============================] - 3s 5ms/step - loss: 0.9792 - accuracy: 0.6105 - val_loss: 0.8085 - val_accuracy: 0.6824
Epoch 6/50
703/703 [==============================] - 3s 5ms/step - loss: 0.8084 - accuracy: 0.6897 - val_loss: 0.7891 - val_accuracy: 0.6887
Epoch 7/50
703/703 [==============================] - 4s 5ms/step - loss: 0.7593 - accuracy: 0.7130 - val_loss: 0.8020 - val_accuracy: 0.6772
Epoch 

In [200]:
# for dense layers
model_b_final = keras.models.clone_model(model_a_final)
model_b_final.build() # replace 10 with number of variables in input layer
model_b_final.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model_b_final.set_weights(model_a_final.get_weights())
prune(model_b_final, 0.0, 0.18)

results = model_b_final.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 1s 6ms/step - loss: 1.3012 - accuracy: 0.6135


In [201]:
total_weights = 0
zero_weights = 0
all_model_weights = model_b_final.get_weights()
for weight in all_model_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

print(results[1], zero_weights / total_weights)
print((0.6135 + zero_weights/total_weights)/2)

0.6134653687477112 0.15993881264830934
0.3867194063241547


In [202]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model_b_final.save_weights("my_model_weights_3.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights_3.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>